In [34]:
import numpy as np
import cv2
from os import listdir
from matplotlib import pyplot as plt
import PIL.Image as pil
import pandas as pd
import math
from scipy.stats import wilcoxon, shapiro 

In [2]:
# Import custom methods
import general_methods
import semantic_segmentation

c:\Users\Kim\anaconda3\envs\pipeline1\lib\site-packages\gluoncv\__init__.py:40: UserWarning: Both `mxnet==1.7.0` and `torch==1.6.0+cpu` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


## Image processing & results gathering

In [3]:
# Get pre-trained models
seg_model = semantic_segmentation.get_pretrained_model()

In [4]:
# Method to perform segmentation and return black image with only walls in grey

def wall_segmentation(filename):
    input_pil = general_methods.import_and_resize(filename)
    input_img = general_methods.import_mx_image(filename)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    size = (width, height)

    # Segmentation
    mmask = semantic_segmentation.get_segementation(input_img, seg_model)
    labels = general_methods.get_labels_string(mmask, mmask.shape[0])
    walls = general_methods.find_colour_indices(labels, "0.47058824,0.47058824,0.47058824,1.0")
    black = np.zeros((height, width, 3))
    segmented_input = semantic_segmentation.remove_inds(black, [walls], [120, 120, 120])

    return segmented_input

In [5]:
# Change seg images so only walls are grey

folder_dir = "images/seg-images/"
 
for image in listdir(folder_dir):
    filename = "images/seg-images/" + image
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    img = input_cv2.copy()
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    for i in range(height):
        for j in range(width):
            if img[i][j][0] == 0:
                img[i][j] = [0, 0, 0]
            if img[i][j][0] != 0:
                img[i][j] = [120, 120, 120]
    
    cv2.imwrite("images/seg-images-after/" + image, img) 

ADE_val_00000079.png
ADE_val_00000085.png
ADE_val_00000091.png
ADE_val_00000093.png
ADE_val_00000101.png
ADE_val_00000141.png
ADE_val_00000151.png
ADE_val_00000160.png
ADE_val_00000245.png
ADE_val_00000303.png
ADE_val_00000314.png
ADE_val_00000316.png
ADE_val_00000318.png
ADE_val_00000420.png
ADE_val_00000423.png


KeyboardInterrupt: 

In [ ]:
for image in listdir("data-collection/groundtruths/segmentation/"):
    filename = "data-collection/groundtruths/segmentation/" + image
    image_name = image[:-13]
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]

    input_pil = pil.open("images/outputs/segmentation/" + image_name + ".jpg")
    input_pil = input_pil.resize((width, height))
    input_pil.save("images/outputs/segmentation-resized/" + image_name + ".jpg")

    input_pil = pil.open("images/seg-images/" + image_name + ".png")
    input_pil = input_pil.resize((width, height))
    input_pil.save("images/seg-images-resized/" + image_name + ".png")

ADE_val_00000079_wall_seg.png


In [ ]:
for image in listdir("images/outputs/segmentation/"):
    filename = "images/outputs/segmentation/" + image
    image_name = image[:-4]
    print(image)
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]

    try:
        img1 = cv2.imread("images/seg-images/" + image_name + ".png")
        img1 = cv2.resize(img1,(width, height),fx=0, fy=0, interpolation = cv2.INTER_NEAREST)
        cv2.imwrite("images/seg-images-resized/" + image_name + ".png", img1)
    except:
        print(image_name + " doesn't exist")

    img2 = cv2.imread("data-collection/groundtruths/segmentation/" + image_name + "_wall_seg.png")
    img2 = cv2.resize(img2,(width, height),fx=0, fy=0, interpolation = cv2.INTER_NEAREST)
    cv2.imwrite("data-collection/groundtruths/segmentation-resized/" + image_name + "_wall_seg.png", img2)

## Intersection over Union

In [ ]:
mine = general_methods.import_cv2_image("images/outputs/segmentation/ADE_val_00000079.jpg")
wiz = general_methods.import_cv2_image("images/seg-images-after/ADE_val_00000079.png")
gt = general_methods.import_cv2_image("data-collection/groundtruths/segmentation-resized/ADE_val_00000079_wall_seg.png")

In [6]:
def get_iuo(image_1, image_2):
    # https://towardsdatascience.com/intersection-over-union-iou-calculation-for-evaluating-an-image-segmentation-model-8b22e2e84686
    intersection = np.logical_and(image_1, image_2)
    union = np.logical_or(image_1, image_2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

In [7]:
df = pd.DataFrame(data=[], columns=["Image_name", "Pipeline_GT_IOU", "Wizart_GT_IOU"])

In [8]:
for image in listdir("images/outputs/segmentation/"):
    filename = "images/outputs/segmentation/" + image
    image_name = image[:-4]
    pl = general_methods.import_cv2_image(filename)
    gt = general_methods.import_cv2_image("data-collection/groundtruths/segmentation-resized/" + image_name + "_wall_seg.png")
    pl_iou = round(get_iuo(gt, pl) * 100, 1)

    try:
        wiz = general_methods.import_cv2_image("images/seg-images-resized/" + image_name + ".png")
        wiz_iou = round(get_iuo(gt, wiz) * 100, 1)
    except:
        wiz_iou = float("nan")

    new_data = [image_name, pl_iou, wiz_iou]
    df.loc[len(df)] = new_data
    
    

In [11]:
# Average wall coverage percentage for both
pipeline_mean = df["Pipeline_GT_IOU"].mean()
wizart_mean = df["Wizart_GT_IOU"].mean()

# Standard deviation for both
pipeline_std = df["Pipeline_GT_IOU"].std()
wizart_std = df["Wizart_GT_IOU"].std()

print(f"pipeline_mean = {pipeline_mean}")
print(f"pipeline_std = {pipeline_std}\n")
print(f"wizart_mean = {wizart_mean}")
print(f"wizart_std = {wizart_std}")

pipeline_mean = 82.79545454545453
pipeline_std = 10.832572640935378

wizart_mean = 76.81052631578947
wizart_std = 13.223112228700257


In [17]:
df.iloc[0]["Pipeline_GT_IOU"]

78.9

In [25]:
# How many was mine better in?
pipeline_count = 0
wizart_count = 0
draw = 0

for i in range(len(df)):
    if (df.iloc[i]["Pipeline_GT_IOU"] > df.iloc[i]["Wizart_GT_IOU"]) or math.isnan(df.iloc[i]["Wizart_GT_IOU"]):
        pipeline_count += 1
    elif df.iloc[i]["Pipeline_GT_IOU"] < df.iloc[i]["Wizart_GT_IOU"]:
        wizart_count += 1
    else:
        draw += 1

print(pipeline_count)
print(wizart_count)
print(draw)

35
9
0


In [42]:
# Drop nan rows
df_new = df.dropna()

pipeline_np = df_new["Pipeline_GT_IOU"].to_numpy()
wizart_np = df_new["Wizart_GT_IOU"].to_numpy()

In [46]:
# Test both for Normality
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html#scipy.stats.shapiro
# If p value less than 0.05 then it's not normally distributed
norm_pipeline = shapiro(pipeline_np)
norm_wizart = shapiro(wizart_np)

print(norm_pipeline)
print(norm_wizart)

# Wilcoxon test (to test significance)
res_1 = wilcoxon(pipeline_np, wizart_np)
print(res_1)
# Significant

ShapiroResult(statistic=0.7533387541770935, pvalue=1.3337198652152438e-06)
ShapiroResult(statistic=0.9200963973999023, pvalue=0.00989960040897131)
WilcoxonResult(statistic=137.0, pvalue=0.0007078925705370894)


In [14]:
df.to_csv("data-collection/outputs/segmentation-iou-2.csv", index=False)